In [13]:
#installing required libs
!pip install langchain langchain-google-genai pypdf faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 7.8 MB/s eta 0:00:00


In [15]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load PDF
loader = PyPDFLoader("attention.pdf")
pages = loader.load()

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(pages)


In [16]:
!pip install -U langchain-community
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load pre-trained sentence transformer model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embedding=embedding_model)


In [17]:
from getpass import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

try:
    api_key = userdata.get("GOOGLE_API_KEY")
except userdata.SecretNotFoundError:
    api_key = getpass("Enter your Gemini API Key: ")


# Load Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", google_api_key=api_key)

Enter your Gemini API Key: ··········


In [18]:
from langchain.chains import RetrievalQA

# Create retriever from FAISS vector store
retriever = vectorstore.as_retriever()

# Set up Retrieval-Augmented Generation chain
rag_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [11]:
# Set up Retrieval-Augmented Generation chain
rag_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [19]:
# Simple interactive chatbot loop in Colab
while True:
    user_query = input("\nYou: ")
    if user_query.lower() in ["exit", "quit"]:
        print("Bot: Goodbye!")
        break
    response = rag_chain.run(user_query)
    print("Bot:", response)



You: hi
Bot: hi

You: what is ai
Bot: I don't have enough information to answer this question.  The provided text focuses on specific papers and publications related to neural networks and machine learning, not on a general definition of AI.

You: what is neural networks
Bot: Based on the provided text, neural networks are computational models inspired by the structure and function of the human brain.  They are used in various machine learning tasks, such as preventing overfitting (Dropout [33]), sequence-to-sequence learning [35], and computer vision [36].  The text also mentions different types of neural networks, including those using convolutional neural networks as building blocks for parallel computation [16, 18, 9].  However, a complete definition of neural networks is not provided.

You: exit
Bot: Goodbye!
